<a href="https://colab.research.google.com/github/belhassen07/Ndevlopi/blob/master/HCP_language.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The HCP dataset comprises task-based fMRI from a large sample of human subjects. The NMA-curated dataset includes time series data that has been preprocessed and spatially-downsampled by aggregating within 360 regions of interest.

In order to use this dataset, please electronically sign the HCP data use terms at ConnectomeDB. Instructions for this are on pp. 24-25 of the HCP Reference Manual.

In this notebook, NMA provides code for downloading the data and doing some basic visualisation and processing.

For a detailed description of the tasks have a look pages 45-54 of the HCP reference manual.

In [ ]:
# @title Install dependencies
!pip install nilearn --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 50.3 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#@title Figure settings
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/course-content/main/nma.mplstyle")

# Basic Parameters

In [ ]:
# The data shared for NMA projects is a subset of the full HCP dataset
N_SUBJECTS = 100

# The data have already been aggregated into ROIs from the Glasser parcellation
N_PARCELS = 360

# The acquisition parameters for all tasks were identical
TR = 0.72  # Time resolution, in seconds

# The parcels are matched across hemispheres with the same order
HEMIS = ["Right", "Left"]

# Each experiment was repeated twice in each subject
RUNS   = ['LR','RL']
N_RUNS = 2

## Experiment Definitions

In [ ]:
# There are 7 tasks. Each has a number of 'conditions'
# TIP: look inside the data folders for more fine-graned conditions

EXPERIMENTS = {
    'MOTOR'      : {'cond':['lf','rf','lh','rh','t','cue']},
    'WM'         : {'cond':['0bk_body','0bk_faces','0bk_places','0bk_tools','2bk_body','2bk_faces','2bk_places','2bk_tools']},
    'EMOTION'    : {'cond':['fear','neut']},
    'GAMBLING'   : {'cond':['loss','win']},
    'LANGUAGE'   : {'cond':['math','story']},
    'RELATIONAL' : {'cond':['match','relation']},
    'SOCIAL'     : {'cond':['ment','rnd']}
}

# Downloading data

## Downloading data file

In [ ]:
import os, requests

fname = "hcp_task.tgz"
url = "https://osf.io/2y3fw/download"

if not os.path.isfile(fname):
  try:
    r = requests.get(url)
  except requests.ConnectionError:
    print("!!! Failed to download data !!!")
  else:
    if r.status_code != requests.codes.ok:
      print("!!! Failed to download data !!!")
    else:
      with open(fname, "wb") as fid:
        fid.write(r.content)

## Extract data

In [ ]:
# The download cells will store the data in nested directories starting here:
HCP_DIR = "./hcp_task"

# importing the "tarfile" module
import tarfile

# open file
with tarfile.open(fname) as tfile:
  # extracting file
  tfile.extractall('.')

subjects = np.loadtxt(os.path.join(HCP_DIR, 'subjects_list.txt'), dtype='str')

# Helper functions

## Function: load_single_timeseries

In [ ]:
def load_single_timeseries(subject, experiment, run, remove_mean=True):
  """Load timeseries data for a single subject and single run.

  Args:
    subject (str):      subject ID to load
    experiment (str):   Name of experiment
    run (int):          (0 or 1)
    remove_mean (bool): If True, subtract the parcel-wise mean (typically the mean BOLD signal is not of interest)

  Returns
    ts (n_parcel x n_timepoint array): Array of BOLD data values

  """
  bold_run  = RUNS[run]
  bold_path = f"{HCP_DIR}/subjects/{subject}/{experiment}/tfMRI_{experiment}_{bold_run}"
  bold_file = "data.npy"
  ts = np.load(f"{bold_path}/{bold_file}")
  if remove_mean:
    ts -= ts.mean(axis=1, keepdims=True)
  return ts

## Function: load_evs

In [ ]:
def load_evs(subject, experiment, run, n_timepoints):
    frames_list = []
    task_key = f'tfMRI_{experiment}_{RUNS[run]}'

    for cond in EXPERIMENTS[experiment]['cond']:
        ev_file  = f"{HCP_DIR}/subjects/{subject}/{experiment}/{task_key}/EVs/{cond}.txt"
        ev_array = np.loadtxt(ev_file, ndmin=2, unpack=True)
        ev       = dict(zip(["onset", "duration", "amplitude"], ev_array))

        start = np.floor(ev["onset"] / TR).astype(int)
        duration = np.ceil(ev["duration"] / TR).astype(int)

        frames = []
        for s, d in zip(start, duration):
            end = s + d
            if s >= n_timepoints:
                continue  # skip out-of-bounds trial
            end = min(end, n_timepoints)
            frames.append(np.arange(s, end))

        frames_list.append(frames)
    return frames_list

## Function: average_frames

In [ ]:
def average_frames(data, evs, experiment, cond):
  idx = EXPERIMENTS[experiment]['cond'].index(cond)
  return np.mean(np.concatenate([np.mean(data[:, evs[idx][i]], axis=1, keepdims=True) for i in range(len(evs[idx]))], axis=-1), axis=1)


## Concatenate Language Data
Code to concatenate language data (2 runs, 316 frames)

In [ ]:
from pathlib import Path
import os

MY_EXPERIMENT = {
    'LANGUAGE'   : {'cond':['math','story']}
}

def concatenate_task_data():
   """
   Concatenate data.npy files for all subjects for each task.
   For each task, concatenate LR and RL runs for each subject, then stack across subjects.

   Returns:
       task_data (dict): Dictionary mapping task names to arrays of shape (n_subjects, n_parcels, n_timepoints_total)
   """

   math_data = []
   story_data = []
   timepoints = 316

   # Iterate through subjects
   for subject in subjects:
       subject_folder = Path(HCP_DIR) / "subjects" / subject
       if not subject_folder.is_dir():
           print(f"Subject directory {subject_folder} does not exist. Skipping.")
           continue

       # Iterate through tasks
       for task in MY_EXPERIMENT:
           task_folder = subject_folder / task
           if not task_folder.is_dir():
               print(f"Task directory {task_folder} does not exist for subject {subject}. Skipping.")
               continue

           # Load and concatenate LR and RL runs
           #subject_task_data = []
           math_activity_data = []
           story_activity_data = []

           for run_idx, run_suffix in enumerate(RUNS):
               try:
                   ts = load_single_timeseries(subject, task, run_idx, remove_mean=False)
                   evs = load_evs(subject=subject, experiment=task, run=run_idx, n_timepoints=timepoints)


                   idx_math = EXPERIMENTS[task]['cond'].index('math')
                   math_activity = np.concatenate([np.mean(ts[:, evs[idx_math][i]], axis=1, keepdims=True) for i in range(len(evs[idx_math]))], axis=-1)
                   #math_activity = average_frames(ts, evs, task, 'math')
                   math_activity_data.append(math_activity)

                   idx_story = EXPERIMENTS[task]['cond'].index('story')
                   story_activity = np.concatenate([np.mean(ts[:, evs[idx_story][i]], axis=1, keepdims=True) for i in range(len(evs[idx_story]))], axis=-1)
                   #story_activity = average_frames(ts, evs, task, 'story')
                   story_activity_data.append(story_activity)

                   #subject_task_data.append(ts)

               except Exception as e:
                   print(f"Error loading data.npy for {subject}/{task}/tfMRI_{task}_{run_suffix}: {e}")
                   #subject_task_data.append(None)
                   math_activity_data.append(None)
                   story_activity_data.append(None)

           # Check if any run failed to load
           if any(x is None for x in math_activity_data):
               print(f"Skipping subject {subject} for task {task} due to missing run data.")
               continue

           if any(x is None for x in story_activity_data):
               print(f"Skipping subject {subject} for task {task} due to missing run data.")
               continue

           # Concatenate LR and RL runs along the time axis (axis=1)
           try:
               #concatenated_ts = np.concatenate(subject_task_data, axis=1)
               #task_data[task].append(concatenated_ts)
                concatenated_math_data = np.concatenate(math_activity_data, axis=1)
                concatenated_story_data = np.concatenate(story_activity_data, axis=1)

                math_data.append(concatenated_math_data)
                story_data.append(concatenated_story_data)
           except Exception as e:
               print(f"Error concatenating runs for {subject}/{task}: {e}")
               continue

   math_stack = np.stack(math_data, axis=0)  # (n_subjects, n_parcels, n_timepoints)
   story_stack = np.stack(story_data, axis=0)

   print(math_stack)
   print(story_stack)

   result = np.vstack([math_stack, story_stack])
   return result

result = concatenate_task_data()
print(result)

ValueError: all input arrays must have the same shape

Designe matrix options:
(200, 360, 316) -- subjects x condition, ROIs, frames
(1600, 360, ?) -- subjects x block, ROIs, frames
(200, 360, 1) -- subjects x concition, ROIs, average frame

# Investigating the Dataset
Investigating data set (JW)


* keep in mind that indices for ROIs now are based on the network and index 1 is a different ROI based on what network you are in
* For exploration/ definition of ROI in the bigger context of our project stick to the indices in the whole data set [0-359]!!!



In [ ]:
regions = np.load(f"{HCP_DIR}/regions.npy").T
region_info = dict(
    name=regions[0].tolist(),
    network=regions[1],
    hemi=['Right']*int(N_PARCELS/2) + ['Left']*int(N_PARCELS/2),
)

## List regions included (Joshua)

In [ ]:
region_names = region_info['name']

for i, name in enumerate(region_names):
    print(f"{i:3d}: {name}")


## Sort regions by network (Joshua)
regions split by network (JW)


*   List item
*   List item

In [ ]:
from collections import defaultdict

# Group regions by network
networks = defaultdict(list)
for name, net in zip(region_info['name'], region_info['network']):
    networks[net].append(name)

# Print all networks and their associated parcels
for network_name in sorted(networks.keys()):
    print(f"\n {network_name} ({len(networks[network_name])} parcels):")
    for i, parcel in enumerate(networks[network_name]):
        print(f"  {i+1:2d}. {parcel}")

## Loading subject info
loading experiment and subject as before

In [ ]:
my_exp = 'LANGUAGE'
my_subj = subjects[1]
my_run = 1

data = load_single_timeseries(subject=my_subj,
                              experiment=my_exp,
                              run=my_run,
                              remove_mean=True)
print(data.shape)

## Amount of Trials per run per Subject (Joshua)
investigate amount of trials per run per subject (JW)


*   probably not most efficient way though

In [ ]:
rows = []

for s in subjects:
    for r, run_name in zip([0, 1], ['LR', 'RL']):
        try:
            data = load_single_timeseries(subject=s, experiment=my_exp, run=r, remove_mean=True)
            evs = load_evs(subject=s, experiment=my_exp, run=r, n_timepoints=data.shape[1])

            if not isinstance(evs, list) or not all(isinstance(x, list) for x in evs):
                raise ValueError(f"EVS returned in wrong format: {type(evs)}")

            row = {
                'subject': s,
                'run': run_name,
                'math': len(evs[0]),
                'story': len(evs[1])
            }

            rows.append(row)

        except Exception as e:
            print(f"Error for subject {s}, run {r}: {e}")

df_trials_runs = pd.DataFrame(rows)

In [ ]:
df_melted = pd.melt(
    df_trials_runs,
    id_vars=['subject', 'run'],
    value_vars=['math', 'story'],
    var_name='condition',
    value_name='n_trials'
)

# Clean up labels
df_melted['condition'] = df_melted['condition'].replace({
    'cond_0_trials': 'math',
    'cond_1_trials': 'story'
})


In [ ]:
df_pivoted = df_trials_runs.pivot(index='subject', columns='run', values=['math', 'story'])

df_pivoted.columns = [f"{run} {cond}" for cond, run in df_pivoted.columns]

df_pivoted.reset_index(inplace=True)

print(df_pivoted.to_string(index=False))


## Update Design Matrix
Code update design matrix (JW):

In [ ]:
conditions = ['math', 'story']
n_rois = 360
rows = [] # stores the BOLD vectors
meta = [] # information about subject, trial, run, condition, ...

for s in subjects:
    for r, run_name in zip([0, 1], ['LR', 'RL']): # Loops over all subjects (s) in the list 'subjects' for both executions of experiment (LR & RL)
        try:
            data = load_single_timeseries(subject=s, experiment=my_exp, run=r, remove_mean=True) # loading timeseries data for a single subject and run (1 row per ROI, 1 column per TR)
            evs = load_evs(subject=s, experiment=my_exp, run=r, n_timepoints=data.shape[1]) # loading evs files/ information about condition, trials, frame indices

            for cond_idx, cond_name in enumerate(conditions):
                for trial_idx, trial_frames in enumerate(evs[cond_idx]): # for each condition gets TR (trials frames) related to that condition
                    # Average BOLD signal across time points (TRs) for each ROI in this trial
                    trial_bold = data[:, trial_frames].mean(axis=1)  # [shape (360,)]
                    rows.append(trial_bold)
                    meta.append({
                        'subject': s,
                        'run': run_name,
                        'condition': cond_name,
                        'trial': trial_idx
                    })

        except Exception as e:
            print(f"Skipping subject {s}, run {r}: {e}")
            continue

# Create final DataFrame
df_trials = pd.DataFrame(rows, columns=[f"ROI_{i}" for i in range(n_rois)])
df_meta = pd.DataFrame(meta)

df_all = pd.concat([df_meta, df_trials], axis=1)

In [ ]:
print(df_all)

In [ ]:
print(df_trials)

In [ ]:
print(df_meta)

## Display mean BOLD activity for a selected region (Joshua)

In [ ]:
region_idx = 165 # this is R_pOFC
region_ts = data[region_idx]


def get_safe_indices(trials, max_len):
   flat = np.concatenate(trials)
   return flat[flat < max_len]


math_idxs = get_safe_indices(evs[0], len(region_ts))
story_idxs = get_safe_indices(evs[1], len(region_ts))


math_vals = np.mean(region_ts[math_idxs])
story_vals = np.mean(region_ts[story_idxs])


plt.bar(['Math', 'Story'], [math_vals, story_vals], color=['steelblue', 'orange'])
plt.ylabel('Mean BOLD signal')
plt.title(f'Region {region_idx}: Math vs Story')
plt.show()

Logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

def model_selection(X, y, C_values):
  """Compute CV accuracy for each C value.
  Args:
    X (2D array): Data matrix
    y (1D array): Label vector
    C_values (1D array): Array of hyperparameter values.
  Returns:
    accuracies (1D array): CV accuracy with each value of C.
  """
  accuracies = []
  for C in C_values:
    # Initialize and fit the model
    # (Hint, you may need to set max_iter)
    model = LogisticRegression(penalty="l2", C=C, max_iter=10000)
    # Get the accuracy for each test split using cross-validation
    accs = cross_val_score(model, X, y, cv=8)
    # Store the average test accuracy for this value of C
    accuracies.append(accs.mean())
  index= np.argmax(accuracies)
  C_max = C_values[index]
  model = LogisticRegression(penalty="l2", C=C_max, max_iter=10000)
  model.fit(X,y)
  return model, C_max


df_trials = pd.DataFrame(rows, columns=[f"ROI_{i}" for i in range(n_rois)])
df_meta = pd.DataFrame(meta)

X = df_trials
y = df_meta["condition"].map({"math": 0, "story": 1}).values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
C_values = np.logspace(-4, 4, 9)

log_model, C_max = model_selection(X_train, y_train, C_values)

In [ ]:
from sklearn.metrics import accuracy_score

y_pred = log_model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

train_preds = log_model.predict(X_train)
train_acc = accuracy_score(y_train, train_preds)
print(train_acc)

In [ ]:
from sklearn.metrics import classification_report

cm1 = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
sns.heatmap(cm1, annot=True, fmt='d', cmap='Blues', xticklabels=['math', 'story'], yticklabels=['math', 'story'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')

print(classification_report(y_train, train_preds))